# Resnet 50

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
image_size = 224
batch_size = 32
train_dir = '/content/drive/MyDrive/Copy of New_Training_resized'  # Replace with your training data directory
test_dir = '/content/drive/MyDrive/New_Validation_resized'    # Replace with your test data directory

# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Loading and augmenting the training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Loading the test data
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Add custom top layers for plant disease classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)  # Output layer based on number of classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Optionally freeze the early layers and fine-tune the later layers
for layer in model.layers[:-4]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for more epochs
model.fit(train_data, epochs=20)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data)
print("Test Accuracy:", test_accuracy)


Found 1717 images belonging to 4 classes.
Found 525 images belonging to 4 classes.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/20
54/54 [==============================] - 481s 9s/step - loss: 1.4019 - accuracy: 0.3436
Epoch 2/20
54/54 [==============================] - 211s 4s/step - loss: 1.3191 - accuracy: 0.3553
Epoch 3/20
54/54 [==============================] - 211s 4s/step - loss: 1.3247 - accuracy: 0.3564
Epoch 4/20
54/54 [==============================] - 212s 4s/step - loss: 1.2916 - accuracy: 0.3593
Epoch 5/20
54/54 [==============================] - 212s 4s/step - loss: 1.2970 - accuracy: 0.3768
Epoch 6/20
54/54 [==============================] - 212s 4s/step - loss: 1.2667 - accuracy: 0.3937
Epoch 7/20
54/54 [==============================] - 210s 4s/step - loss: 1.2896 - accuracy: 0.3867
Epoch 8/20
54/54 [==============================] - 209s 4s/step - loss: 1.2817 - accuracy: 0.3885
Epoch 9/20
54/54 [==============================] - 209s 4s/